In [ ]:
import sys, time, os, asyncio, glob, re
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
dates = ["2023-08-29","2023-09-14","2023-09-26","2023-09-28"]
dir = "/scratch/labJackData/"
files = os.listdir(dir)
azs = []

for date in dates:
    start = Time(f"{date}T16:00:00Z", scale='utc')
    date_split = date.split('-')
    end_date = f"{date_split[0]}-{date_split[1]}-{int(date_split[2])+1:02}"
    print(date, end_date)
    end = Time(f"{end_date}T10:00:00Z", scale='utc')

    dome = await client.select_time_series('lsst.sal.ATDome.position', \
                                                        ['azimuthPosition'],  start, end)
    strippedDate = date.replace('-','')
    nextStrippedDate = str(int(strippedDate) + 1)
    counter = 0
    for file in files:
        split_file = re.split('_|T|\Z.', file)
        if split_file[0] == "Dome":
            if (split_file[1] == nextStrippedDate) or \
                (split_file[1] == strippedDate and int(list(split_file[2])[0]) > 1):
                filename = dir+file

                pkl_file = open(filename, 'rb')
                df = pkl.load(pkl_file)
                pkl_file.close()
                #print(filename, len(df), df.columns)
                if counter == 0:
                    merged_df = df
                else:
                    merged_df = pd.concat([merged_df, df])
                #if counter == 2:
                #    break
                counter += 1
    print(date, counter)

    close_df = merged_df[merged_df['AIN1'] > 6.0]
    start_times = []
    end_times = []
    durations = []
    for i in range(len(close_df)):
        if i == 0:
            start_time = close_df.index[0]
            continue
        if Time(close_df.index[i].isoformat()).unix_tai - Time(close_df.index[i-1].isoformat()).unix_tai < 0.05:
            continue
        else:
            end_time = close_df.index[i-1]
            duration = Time(end_time.isoformat()).unix_tai - Time(start_time.isoformat()).unix_tai
            if duration < 1.0:
                #print(start_time, end_time, duration)
                start_times.append(start_time)
                end_times.append(end_time)
                durations.append(duration)
                subDome = dome[dome.index < pd.Timestamp(start_time, tz='UTC')]
                az = subDome.values[-1][0]
                azs.append(az)
                #print(start_time, end_time, duration, az)
            start_time = close_df.index[i]
    print(date, len(start_times))


In [ ]:
plt.title("Histogram of dome azimuth for phantom close events\n2023-08-29 through 2023-09-29")
plt.hist(azs, bins=45)
plt.xlim(0,360)
plt.text(200,15,f"{len(azs)} events")
plt.xlabel("Dome azimuth (degrees)")
plt.savefig("/home/craiglagegit/DATA/Phantom_Close_Azimuths_13Oct23.png")

In [ ]:
dates = ["2023-08-29","2023-09-14","2023-09-26","2023-09-28"]
dir = "/scratch/labJackData/"
files = os.listdir(dir)
fig, axs = plt.subplots(len(dates),1,figsize=(12,12))
plt.subplots_adjust(hspace=1.0)

for n, date in enumerate(dates):
    start = Time(f"{date}T16:00:00Z", scale='utc')
    date_split = date.split('-')
    end_date = f"{date_split[0]}-{date_split[1]}-{int(date_split[2])+1:02}"
    print(date, end_date)
    end = Time(f"{end_date}T10:00:00Z", scale='utc')
    dome = await client.select_time_series('lsst.sal.ATDome.position', \
                                                        ['azimuthPosition'],  start, end)
    move = await client.select_time_series('lsst.sal.ATDome.command_moveAzimuth', \
                                                        ['azimuth'],  start, end)
    move_times = move.index
    move_times_pos = []
    move_times_neg = []
    for time in move_times:
        subDome = dome[dome.index > time]
        if subDome.values[10] > subDome.values[0]:
            move_times_pos.append(time)
        if subDome.values[10] < subDome.values[0]:
            move_times_neg.append(time)

    strippedDate = date.replace('-','')
    nextStrippedDate = str(int(strippedDate) + 1)
    counter = 0
    for file in files:
        split_file = re.split('_|T|\Z.', file)
        if split_file[0] == "Dome":
            if (split_file[1] == nextStrippedDate) or \
                (split_file[1] == strippedDate and int(list(split_file[2])[0]) > 1):
                filename = dir+file

                pkl_file = open(filename, 'rb')
                df = pkl.load(pkl_file)
                pkl_file.close()
                #print(filename, len(df), df.columns)
                if counter == 0:
                    merged_df = df
                else:
                    merged_df = pd.concat([merged_df, df])
                #if counter == 2:
                #    break
                counter += 1
    print(date, counter)

    close_df = merged_df[merged_df['AIN1'] > 6.0]
    start_times = []
    end_times = []
    durations = []
    for i in range(len(close_df)):
        if i == 0:
            start_time = close_df.index[0]
            continue
        if Time(close_df.index[i].isoformat()).unix_tai - Time(close_df.index[i-1].isoformat()).unix_tai < 0.05:
            continue
        else:
            end_time = close_df.index[i-1]
            duration = Time(end_time.isoformat()).unix_tai - Time(start_time.isoformat()).unix_tai
            if duration < 1.0:
                start_times.append(start_time)
                end_times.append(end_time)
                durations.append(duration)
            start_time = close_df.index[i]
    print(date, len(start_times))
    ax = axs[n]
    ax.set_title(f"Phantom close events - {date} \n Black-close event, Green - Move Pos, Red - Move neg")
    ax.set_ylabel("Dome azimuth (degrees)")
    #ax.set_xlabel("Time")
    dome['azimuthPosition'].plot(ax=ax, color='b')
    for start_time in start_times:
        ax.axvline(start_time, color='k')
    for time in move_times_pos:
        ax.axvline(time, color='g')
    for time in move_times_neg:
        ax.axvline(time, color='r')
    ax.set_xlim(start_times[0]-pd.Timedelta(minutes=10), start_times[-1]+pd.Timedelta(minutes=10))

plt.savefig("/home/craiglagegit/DATA/Phantom_Close_Movements_16Oct23.png")

In [ ]:
dates = ["2023-08-29","2023-09-14","2023-09-26","2023-09-28"]
dir = "/scratch/labJackData/"
files = os.listdir(dir)
fig, ax = plt.subplots(1,1,figsize=(12,5))
plt.subplots_adjust(hspace=1.0)

for n, date in enumerate([dates[0]]):
    start = Time(f"{date}T16:00:00Z", scale='utc')
    date_split = date.split('-')
    end_date = f"{date_split[0]}-{date_split[1]}-{int(date_split[2])+1:02}"
    print(date, end_date)
    end = Time(f"{end_date}T10:00:00Z", scale='utc')
    dome = await client.select_time_series('lsst.sal.ATDome.position', \
                                                        ['azimuthPosition'],  start, end)
    move = await client.select_time_series('lsst.sal.ATDome.command_moveAzimuth', \
                                                        ['azimuth'],  start, end)
    move_times = move.index
    move_times_pos = []
    move_times_neg = []
    for time in move_times:
        subDome = dome[dome.index > time]
        if subDome.values[10] > subDome.values[0]:
            move_times_pos.append(time)
        if subDome.values[10] < subDome.values[0]:
            move_times_neg.append(time)

    strippedDate = date.replace('-','')
    nextStrippedDate = str(int(strippedDate) + 1)
    counter = 0
    for file in files:
        split_file = re.split('_|T|\Z.', file)
        if split_file[0] == "Dome":
            if (split_file[1] == nextStrippedDate) or \
                (split_file[1] == strippedDate and int(list(split_file[2])[0]) > 1):
                filename = dir+file

                pkl_file = open(filename, 'rb')
                df = pkl.load(pkl_file)
                pkl_file.close()
                #print(filename, len(df), df.columns)
                if counter == 0:
                    merged_df = df
                else:
                    merged_df = pd.concat([merged_df, df])
                #if counter == 2:
                #    break
                counter += 1
    print(date, counter)

    close_df = merged_df[merged_df['AIN1'] > 6.0]
    start_times = []
    end_times = []
    durations = []
    for i in range(len(close_df)):
        if i == 0:
            start_time = close_df.index[0]
            continue
        if Time(close_df.index[i].isoformat()).unix_tai - Time(close_df.index[i-1].isoformat()).unix_tai < 0.05:
            continue
        else:
            end_time = close_df.index[i-1]
            duration = Time(end_time.isoformat()).unix_tai - Time(start_time.isoformat()).unix_tai
            if duration < 1.0:
                start_times.append(start_time)
                end_times.append(end_time)
                durations.append(duration)
            start_time = close_df.index[i]
    print(date, len(start_times))
    #ax = axs[n]
    ax.set_title(f"Phantom close events - {date} \n Black-close event, Green - Move Pos, Red - Move neg")
    ax.set_ylabel("Dome azimuth (degrees)")
    #ax.set_xlabel("Time")
    dome['azimuthPosition'].plot(ax=ax, color='b')
    for start_time in start_times:
        ax.axvline(start_time, color='k')
    for time in move_times_pos:
        ax.axvline(time, color='g')
    for time in move_times_neg:
        ax.axvline(time, color='r')
    ax.set_xlim(Time("2023-08-30T01:30:00").isot, Time("2023-08-30T02:00:00").isot)
    ax.set_ylim(285,290)

plt.savefig("/home/craiglagegit/DATA/Phantom_Close_Movements_BlowUp_16Oct23.png")